# Bigfive Exploration

In [27]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Python version:", sys.version)
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)
print("Seaborn version:", sns.__version__)

Python version: 3.12.0 (v3.12.0:0fb18b02c8, Oct  2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]
Pandas version: 2.3.0
Numpy version: 2.3.1
Seaborn version: 0.13.2


In [28]:
#Import dataset
df = pd.read_csv("../data/raw/raw_data.csv", sep='\t')

### Data Overview

In [29]:
df.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0


In [30]:
df.shape

(1015341, 110)

## Data Cleaning

#### Keep unique participants by removing duplicates by IP address


In [31]:
#Multiple IP counts could be the same person doing the test multiple times  
df[df['IPC'] != 1].shape

(318496, 110)

In [32]:
#Keep unique participants by removing duplicates by IP address
df_clean = df[df['IPC'] == 1]
df_clean.shape

(696845, 110)

#### Drop rows with any missing values in the 50 questions


In [33]:
#Drop rows with any missing values in the 50 questions
# Check total missing per question
import re

item_cols = [col for col in df_clean.columns if re.match(r'^(EXT|EST|AGR|CSN|OPN)\d+$', col)]
print(item_cols)
print(len(item_cols))  # Should be exactly 50


['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9', 'EXT10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8', 'EST9', 'EST10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7', 'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6', 'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5', 'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10']
50


#### Drop rows with any invalid responses (score outside of 1-5) 


In [34]:
print(df_clean[item_cols].min().min())
print(df_clean[item_cols].max().max())


0.0
5.0


In [35]:
#Drop rows with any invalid responses (score outside of 1-5) 
invalid_score = (df_clean[item_cols] < 1) | (df_clean[item_cols] > 5)
print("Total invalid entries:", invalid_score.sum().sum())
rows_with_invalid = invalid_score.any(axis=1)
print("Rows with at least one invalid entry:", rows_with_invalid.sum())

Total invalid entries: 234989
Rows with at least one invalid entry: 92382


In [36]:
df_clean = df_clean[~rows_with_invalid]
print(df_clean.shape)

(604463, 110)


#### Drop unrealistic testelapse values

In [37]:
#testelapse = the time(s) spent answering the 50 questions
#Finishing 50 questions under 20 seconds is near-impossible -> could be bots
df_clean = df_clean[df_clean['testelapse'] >= 20]
print(df_clean.shape)

(603223, 110)


#### Drop rows with introelapse == 0

In [38]:
#introelapse = time spent on the landing page -> shouldn't be 0
df_clean = df_clean[df_clean['introelapse'] > 0]
print(df_clean.shape)

(602587, 110)


#### Filter by Country [Optional]

In [39]:
#Filtering by Country lines for possible future use
#Selecting only Canadian entries
df_CA = df_clean[df_clean['country'].isin(['CA'])]
print(df_CA['country'].value_counts())
print(df_CA.shape)

country
CA    38329
Name: count, dtype: int64
(38329, 110)


##### Calculate mean score for each trait


In [40]:
# Calculate mean score for each trait
df_clean['Extraversion'] = df_clean[[f'EXT{i}' for i in range(1, 11)]].mean(axis=1)
df_clean['Emotional Stability'] = df_clean[[f'EST{i}' for i in range(1, 11)]].mean(axis=1)
df_clean['Agreeableness'] = df_clean[[f'AGR{i}' for i in range(1, 11)]].mean(axis=1)
df_clean['Conscientiousness'] = df_clean[[f'CSN{i}' for i in range(1, 11)]].mean(axis=1)
df_clean['Openness'] = df_clean[[f'OPN{i}' for i in range(1, 11)]].mean(axis=1)

df_clean[['Extraversion', 'Emotional Stability', 'Agreeableness', 'Conscientiousness', 'Openness']].head()

,Extraversion,Emotional Stability,Agreeableness,Conscientiousness,Openness
0,3.0,2.4,3.1,3.2,3.3
1,3.4,2.1,3.2,3.1,2.7
2,2.9,2.6,2.8,2.8,3.1
3,2.6,2.7,3.2,2.7,3.1
5,3.0,2.2,2.8,2.9,3.2


In [41]:
df_clean[['Extraversion', 'Emotional Stability', 'Agreeableness', 'Conscientiousness', 'Openness']].describe()

,Extraversion,Emotional Stability,Agreeableness,Conscientiousness,Openness
count,602587.000000,602587.000000,602587.000000,602587.000000,602587.000000
mean,3.030420,3.047360,3.167351,3.143114,3.309093
std,0.332876,0.641163,0.345323,0.375442,0.365255
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.800000,2.600000,3.000000,2.900000,3.100000
50%,3.000000,3.000000,3.200000,3.100000,3.300000
75%,3.200000,3.500000,3.400000,3.400000,3.600000
max,5.000000,5.000000,5.000000,5.000000,5.000000


#### Reverse Coding Emotional Stability to Neuroticism

In [ ]:

est_items = [col for col in df_clean.columns if col.startswith('EST') and not col.endswith('_E')]
est_items

['EST1',
 'EST2',
 'EST3',
 'EST4',
 'EST5',
 'EST6',
 'EST7',
 'EST8',
 'EST9',
 'EST10']

In [ ]:
#Reverse coding emotional stability items to neuroticism items
for i in range(1, 11):
    df_clean[f'NRT{i}'] = 6 - df_clean[f'EST{i}']

In [ ]:
#Checking if the NRT columns are created 
nrt_items = [col for col in df_clean.columns if col.startswith('NRT')]
nrt_items

['NRT1',
 'NRT2',
 'NRT3',
 'NRT4',
 'NRT5',
 'NRT6',
 'NRT7',
 'NRT8',
 'NRT9',
 'NRT10']

In [47]:
#Calculate mean score for Neuroticism
df_clean['Neuroticism'] = df_clean[[f'NRT{i}' for i in range(1, 11)]].mean(axis=1)


In [48]:
#Comparing EST and NRT
df_clean[['Neuroticism', 'Emotional Stability']]

,Neuroticism,Emotional Stability
0,3.6,2.4
1,3.9,2.1
2,3.4,2.6
3,3.3,2.7
5,3.8,2.2
...,...,...
1015334,3.2,2.8
1015335,1.8,4.2
1015337,2.0,4.0
1015339,2.7,3.3


#### Saving Cleaned Data in CSV

In [49]:
df_clean.to_csv('../data/cleaned/cleaned_bigfive_v1.csv', index=False)